In [1]:
from flask import Flask, render_template
import pandas as pd
import numpy as np
from queue import PriorityQueue

In [2]:
manifest_name = "SS TEST SHIP"
employee_name = "John Doe"
nrows=8
ncols=12

manifest = []
for case in range(1,6):
    manifest.append( pd.read_csv('./ship_cases/ShipCase'+str(case)+'.txt',header=None,names=["col","row","weight","name"]) )
    manifest[case-1]['col'] = manifest[case-1]['col'].str.replace("[","", regex=True)
    manifest[case-1]['row'] = manifest[case-1]['row'].str.replace("]","", regex=True)
    manifest[case-1]['weight'] = manifest[case-1]['weight'].str.replace("{","", regex=True).replace("}","", regex=True)

In [3]:
testManifest = ( pd.read_csv('./ship_cases/ShipCase2.txt',header=None,names=["col","row","weight","cont"]) )
testManifest['col'] = testManifest['col'].str.replace("\[| ","", regex=True)
testManifest['row'] = testManifest['row'].str.replace("\]| ","", regex=True)
testManifest['weight'] = testManifest['weight'].str.replace("{|}| ","", regex=True)
testManifest['cont'] = (testManifest['cont'].str[1:])

testManifest.head()

,col,row,weight,cont
0,01,01,00000,NAN
1,01,02,00000,NAN
2,01,03,00000,NAN
3,01,04,00120,Ram
4,01,05,00000,UNUSED


In [4]:
class Cell:
    def __init__(self, col, row, weight, cont):
        self.col = col
        self.row = row
        self.weight = weight
        seld.cont = cont
#not actually using this

In [5]:
#creating 2d np array for ship
testManifestArray = np.array(testManifest.loc[:]).reshape(nrows,ncols,4)
print(testManifestArray[0,1])

['01' '02' '00000' 'NAN']


In [6]:
#creating 2d np array for buffer
testBuffer = np.zeros(4*24).reshape(4,24)
print(testBuffer)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


####

In [7]:
def printState(array):
    for j in range(7,-1,-1):
        print("")
        for i in array[j]:
            if(i[3]=="UNUSED"):
                print("[  ]", end="\t")
            else:
                print(i[3], end="\t")
#printState(testManifestArray)

In [8]:
printState(testManifestArray)


[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
Cat	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
NAN	Dog	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	NAN	
NAN	NAN	NAN	Ram	[  ]	[  ]	[  ]	[  ]	Owl	NAN	NAN	NAN	

### Defining a State

In [9]:
#Need Matrix of Ship Containers
#Need Matrix of Buffer Zone
#Need Col # the Crane is picking from
class State:
    def __init__(self, ship, buffer=testBuffer):
        self.ship = ship
        self.buffer = buffer
        #self.colPos = colPos

    def __str__(self):
        printState(self.ship)
        #return f"\n\nPick Col: {self.colPos}"
        return ""

In [10]:
#decorator clas for use with branching
class PickState:
    def __init__(self, state, colPick):
        self.state = state
        self.colPick = colPick

    def __str__(self):
        printState(self.state.ship)
        return f"\n\nPick Col: {self.colPick}"

In [11]:
print(PickState(State(testManifestArray), 1))


[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
Cat	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
NAN	Dog	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	NAN	
NAN	NAN	NAN	Ram	[  ]	[  ]	[  ]	[  ]	Owl	NAN	NAN	NAN	

Pick Col: 1


### Branching Factor is 12 (number of col)

In [12]:
#To branch we place our picked container (top most of selected col) on top of highest container of another col


#### Helper Function to find Highest Container in Stack

In [13]:
def stackHeight(array, col, nrows=nrows):
    for row in range(nrows-1,-1,-1):
        if(array[row,col,3] != "UNUSED"):
            return row
    return(-1)
    #-1 means no containers/blocked spaces in col

In [14]:
testCol=4
print(testManifestArray[:,testCol])
stackHeight(testManifestArray,testCol)

[['01' '05' '00000' 'UNUSED']
 ['02' '05' '00000' 'UNUSED']
 ['03' '05' '00000' 'UNUSED']
 ['04' '05' '00000' 'UNUSED']
 ['05' '05' '00000' 'UNUSED']
 ['06' '05' '00000' 'UNUSED']
 ['07' '05' '00000' 'UNUSED']
 ['08' '05' '00000' 'UNUSED']]


-1

#### Helper to find if can pick container

In [15]:
def pickable(array, col, nrows=nrows):
    h = stackHeight(array,col,nrows)
    if(array[h,col,3]=="NAN" or h==-1):
        return False
    return True

#### Place container to new col

In [16]:
def moveContainer(array, col, newCol):
    if(col==newCol):
        return -1
    #find where to place container
    h1 = stackHeight(array,col)
    h2 = stackHeight(array,newCol) +1 
        
    newArray = array.copy()
    newArray[h2,newCol,2:4] = newArray[h1,col,2:4]  
    newArray[h1,col,2:4] = ['00000','UNUSED']
    
#     print((newArray[h1]))
    return newArray

print(State(testManifestArray))
print()
print(State(moveContainer(testManifestArray,1,2)))


[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
Cat	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
NAN	Dog	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	NAN	
NAN	NAN	NAN	Ram	[  ]	[  ]	[  ]	[  ]	Owl	NAN	NAN	NAN	


[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
Cat	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
NAN	[  ]	Dog	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	NAN	
NAN	NAN	NAN	Ram	[  ]	[  ]	[  ]	[  ]	Owl	NAN	NAN	NAN	


## Balancing Problem

In [17]:
#Function for testing balance
def balanceScore(array, cols=ncols):
    split=cols/2
    left = sum(((array[:,:int(split),2]).flatten()).astype(int))
    #print(left)
    right = sum(((array[:,int(split):,2]).flatten()).astype(int))
    #print(right)
    
    if(left==right):
        return 1
    if(left==0 or right==0):
        return 0
    
    return min(left,right)/max(left,right)
#balanceScore(testManifestArray)

In [18]:
#Function for accepting balance
#default threshold of 10% 
def balanced(array, cols=ncols, thresh=.1):
    if(balanceScore(array, cols) > (1-thresh)):
        return True
    return False
#balanced(testManifestArray)

### Priority Queue Helper

In [19]:
#Code Chunk from official python priority queue page
from dataclasses import dataclass, field
from typing import Any

@dataclass(order=True)
class PrioritizedItem:
    priority: int
    item: Any=field(compare=False)

### Heuristic (TODO)

In [20]:
def totalWeight(array):
    #array=state.state.ship
    return int(1)
#unfinished

In [21]:
PickState(State(testManifestArray),1)

### Branching Loop

In [22]:
#initial state:
initialState = State(testManifestArray)

#duplicate states checker
duplicateState = []

#legal final states
finalState = []

#Create PriorityQueue for queing states
queueP = PriorityQueue()
queueD = PriorityQueue()

#queue.put((0,initialState))

#Choose which container to move
    #creating and adding those states
for i in range(0,ncols):
    newPickState = PickState(initialState,i)
    if(pickable(newPickState.state.ship,i)):
        print("State with col position:",i,"appended:")
        queueP.put(PrioritizedItem(totalWeight(newPickState),newPickState))
        if(newPickState.state not in duplicateState):
            duplicateState.append(newPickState.state)

        
depth = 0

#Choose where to move the container
    #creating and adding those states
while True:
    depth +=1
    while not queueP.empty():
        currentState = queueP.get().item
        if(balanced(currentState.state.ship)):
            print("Already Balanced")
            finalState.append(currentState)
            break;
        #Create new branch
        for newCol in range(0,ncols):
            #try to place container in each col
            if(stackHeight(currentState.state.ship,currentState.colPick)<nrows and newCol!=currentState.colPick):
                #does not allow if overflow or where it already was
                newDropState = State(moveContainer(currentState.state.ship,currentState.colPick,newCol))
                if((newDropState not in duplicateState)):
                    print("Creating new dropped state:", newCol)
                    duplicateState.append(newDropState)
                    queueD.put(PrioritizedItem(totalWeight(newDropState.ship),newDropState))

    while not queueD.empty():
        currentState=queueD.get().item
        if(balanced(currentState.ship)):
            print("Already Balanced")
            finalState.append(currentState)
            break;
    #Choose which container to move
        #creating and adding those states
        for w in range(0,ncols):
            newPickState = PickState(currentState,w)
            if(pickable(newPickState.state.ship,w)):
                print("State with col position:",w,"appended:")
                queueP.put(PrioritizedItem(totalWeight(newPickState),newPickState))

    if depth>=20:
        break

State with col position: 0 appended:
State with col position: 1 appended:
State with col position: 3 appended:
State with col position: 8 appended:
Creating new dropped state: 1
Creating new dropped state: 2
Creating new dropped state: 3
Creating new dropped state: 4
Creating new dropped state: 5
Creating new dropped state: 6
Creating new dropped state: 7
Creating new dropped state: 8
Creating new dropped state: 9
Creating new dropped state: 10
Creating new dropped state: 11
Creating new dropped state: 0
Creating new dropped state: 1
Creating new dropped state: 2
Creating new dropped state: 4
Creating new dropped state: 5
Creating new dropped state: 6
Creating new dropped state: 7
Creating new dropped state: 8
Creating new dropped state: 9
Creating new dropped state: 10
Creating new dropped state: 11
Creating new dropped state: 0
Creating new dropped state: 2
Creating new dropped state: 3
Creating new dropped state: 4
Creating new dropped state: 5
Creating new dropped state: 6
Creating

Creating new dropped state: 10
Creating new dropped state: 11
Creating new dropped state: 0
Creating new dropped state: 1
Creating new dropped state: 2
Creating new dropped state: 4
Creating new dropped state: 5
Creating new dropped state: 6
Creating new dropped state: 7
Creating new dropped state: 8
Creating new dropped state: 9
Creating new dropped state: 10
Creating new dropped state: 11
Creating new dropped state: 0
Creating new dropped state: 2
Creating new dropped state: 3
Creating new dropped state: 4
Creating new dropped state: 5
Creating new dropped state: 6
Creating new dropped state: 7
Creating new dropped state: 8
Creating new dropped state: 9
Creating new dropped state: 10
Creating new dropped state: 11
Creating new dropped state: 0
Creating new dropped state: 1
Creating new dropped state: 2
Creating new dropped state: 3
Creating new dropped state: 4
Creating new dropped state: 5
Creating new dropped state: 6
Creating new dropped state: 7
Creating new dropped state: 9
Crea

Creating new dropped state: 10
Creating new dropped state: 11
Creating new dropped state: 1
Creating new dropped state: 2
Creating new dropped state: 3
Creating new dropped state: 4
Creating new dropped state: 5
Creating new dropped state: 6
Creating new dropped state: 7
Creating new dropped state: 8
Creating new dropped state: 9
Creating new dropped state: 10
Creating new dropped state: 11
Creating new dropped state: 0
Creating new dropped state: 1
Creating new dropped state: 2
Creating new dropped state: 3
Creating new dropped state: 4
Creating new dropped state: 5
Creating new dropped state: 6
Creating new dropped state: 7
Creating new dropped state: 8
Creating new dropped state: 10
Creating new dropped state: 11
Creating new dropped state: 0
Creating new dropped state: 1
Creating new dropped state: 2
Creating new dropped state: 3
Creating new dropped state: 4
Creating new dropped state: 5
Creating new dropped state: 6
Creating new dropped state: 7
Creating new dropped state: 9
Crea

Creating new dropped state: 2
Creating new dropped state: 3
Creating new dropped state: 4
Creating new dropped state: 5
Creating new dropped state: 6
Creating new dropped state: 7
Creating new dropped state: 8
Creating new dropped state: 9
Creating new dropped state: 10
Creating new dropped state: 11
Creating new dropped state: 0
Creating new dropped state: 1
Creating new dropped state: 2
Creating new dropped state: 3
Creating new dropped state: 4
Creating new dropped state: 5
Creating new dropped state: 6
Creating new dropped state: 7
Creating new dropped state: 8
Creating new dropped state: 9
Creating new dropped state: 11
Creating new dropped state: 0
Creating new dropped state: 1
Creating new dropped state: 2
Creating new dropped state: 3
Creating new dropped state: 4
Creating new dropped state: 5
Creating new dropped state: 6
Creating new dropped state: 7
Creating new dropped state: 9
Creating new dropped state: 10
Creating new dropped state: 11
Creating new dropped state: 0
Creat

Creating new dropped state: 8
Creating new dropped state: 9
Creating new dropped state: 10
Creating new dropped state: 11
Creating new dropped state: 0
Creating new dropped state: 2
Creating new dropped state: 3
Creating new dropped state: 4
Creating new dropped state: 5
Creating new dropped state: 6
Creating new dropped state: 7
Creating new dropped state: 8
Creating new dropped state: 9
Creating new dropped state: 10
Creating new dropped state: 11
Creating new dropped state: 1
Creating new dropped state: 2
Creating new dropped state: 3
Creating new dropped state: 4
Creating new dropped state: 5
Creating new dropped state: 6
Creating new dropped state: 7
Creating new dropped state: 8
Creating new dropped state: 9
Creating new dropped state: 10
Creating new dropped state: 11
Creating new dropped state: 0
Creating new dropped state: 1
Creating new dropped state: 2
Creating new dropped state: 3
Creating new dropped state: 4
Creating new dropped state: 5
Creating new dropped state: 6
Crea

...

Creating new dropped state: 2
Creating new dropped state: 3
Creating new dropped state: 4
Creating new dropped state: 5
Creating new dropped state: 6
Creating new dropped state: 7
Creating new dropped state: 8
Creating new dropped state: 9
Creating new dropped state: 10
Creating new dropped state: 11
Creating new dropped state: 1
Creating new dropped state: 2
Creating new dropped state: 3
Creating new dropped state: 4
Creating new dropped state: 5
Creating new dropped state: 6
Creating new dropped state: 7
Creating new dropped state: 8
Creating new dropped state: 9
Creating new dropped state: 10
Creating new dropped state: 11
Creating new dropped state: 0
Creating new dropped state: 1
Creating new dropped state: 2
Creating new dropped state: 3
Creating new dropped state: 4
Creating new dropped state: 5
Creating new dropped state: 6
Creating new dropped state: 7
Creating new dropped state: 9
Creating new dropped state: 10
Creating new dropped state: 11
Creating new dropped state: 0
Crea

Creating new dropped state: 9
Creating new dropped state: 11
Creating new dropped state: 0
Creating new dropped state: 1
Creating new dropped state: 2
Creating new dropped state: 3
Creating new dropped state: 4
Creating new dropped state: 5
Creating new dropped state: 6
Creating new dropped state: 7
Creating new dropped state: 9
Creating new dropped state: 10
Creating new dropped state: 11
Creating new dropped state: 0
Creating new dropped state: 1
Creating new dropped state: 2
Creating new dropped state: 3
Creating new dropped state: 4
Creating new dropped state: 5
Creating new dropped state: 6
Creating new dropped state: 8
Creating new dropped state: 9
Creating new dropped state: 10
Creating new dropped state: 11
Creating new dropped state: 1
Creating new dropped state: 2
Creating new dropped state: 3
Creating new dropped state: 4
Creating new dropped state: 5
Creating new dropped state: 6
Creating new dropped state: 7
Creating new dropped state: 8
Creating new dropped state: 9
Creat

Creating new dropped state: 5
Creating new dropped state: 6
Creating new dropped state: 7
Creating new dropped state: 8
Creating new dropped state: 9
Creating new dropped state: 10
Creating new dropped state: 11
Creating new dropped state: 1
Creating new dropped state: 2
Creating new dropped state: 3
Creating new dropped state: 4
Creating new dropped state: 5
Creating new dropped state: 6
Creating new dropped state: 7
Creating new dropped state: 8
Creating new dropped state: 9
Creating new dropped state: 10
Creating new dropped state: 11
Creating new dropped state: 0
Creating new dropped state: 1
Creating new dropped state: 2
Creating new dropped state: 3
Creating new dropped state: 4
Creating new dropped state: 5
Creating new dropped state: 6
Creating new dropped state: 8
Creating new dropped state: 9
Creating new dropped state: 10
Creating new dropped state: 11
Creating new dropped state: 0
Creating new dropped state: 1
Creating new dropped state: 2
Creating new dropped state: 3
Crea

Creating new dropped state: 9
Creating new dropped state: 10
Creating new dropped state: 11
Creating new dropped state: 0
Creating new dropped state: 1
Creating new dropped state: 2
Creating new dropped state: 3
Creating new dropped state: 4
Creating new dropped state: 5
Creating new dropped state: 6
Creating new dropped state: 8
Creating new dropped state: 9
Creating new dropped state: 10
Creating new dropped state: 11
Creating new dropped state: 0
Creating new dropped state: 1
Creating new dropped state: 2
Creating new dropped state: 3
Creating new dropped state: 4
Creating new dropped state: 5
Creating new dropped state: 6
Creating new dropped state: 8
Creating new dropped state: 9
Creating new dropped state: 10
Creating new dropped state: 11
Creating new dropped state: 0
Creating new dropped state: 1
Creating new dropped state: 2
Creating new dropped state: 3
Creating new dropped state: 4
Creating new dropped state: 6
Creating new dropped state: 7
Creating new dropped state: 8
Crea

Creating new dropped state: 2
Creating new dropped state: 3
Creating new dropped state: 4
Creating new dropped state: 6
Creating new dropped state: 7
Creating new dropped state: 8
Creating new dropped state: 9
Creating new dropped state: 10
Creating new dropped state: 11
Creating new dropped state: 0
Creating new dropped state: 1
Creating new dropped state: 3
Creating new dropped state: 4
Creating new dropped state: 5
Creating new dropped state: 6
Creating new dropped state: 7
Creating new dropped state: 8
Creating new dropped state: 9
Creating new dropped state: 10
Creating new dropped state: 11
Creating new dropped state: 0
Creating new dropped state: 2
Creating new dropped state: 3
Creating new dropped state: 4
Creating new dropped state: 5
Creating new dropped state: 6
Creating new dropped state: 7
Creating new dropped state: 8
Creating new dropped state: 9
Creating new dropped state: 10
Creating new dropped state: 11
Creating new dropped state: 0
Creating new dropped state: 1
Crea

In [23]:
# for x in duplicateState:
#     print(x,"\n")

In [24]:
for x in finalState:
    print(x,"\n")


[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
NAN	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	Dog	[  ]	NAN	
NAN	NAN	NAN	Ram	[  ]	[  ]	[  ]	Cat	Owl	NAN	NAN	NAN	 


[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	Dog	
NAN	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	NAN	
NAN	NAN	NAN	Ram	[  ]	[  ]	[  ]	Cat	Owl	NAN	NAN	NAN	 


[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[